In [1]:
import pyspark

In [2]:
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark import SparkContext

In [3]:
conf = SparkConf().setAll([('spark.driver.extraClassPath',
                            '/Users/go/Downloads/mysql-connector-java-5.1.47-bin.jar')])  # jar path추가.

sc = SparkContext(appName="TestPySparkJDBC", conf=conf)
sqlContext = SQLContext(sc)
 
hostname = "localhost"
dbname = "testdb"
jdbcPort = 3306
username = "root"
password = "root"
jdbc_url = "jdbc:mysql://{0}:{1}/{2}?user={3}&password={4}&useSSL=false&allowPublicKeyRetrieval=true".format(hostname,jdbcPort,dbname,username,password)
 
query = '(select 1) AS t'


In [4]:
df1 = sqlContext.read.format('jdbc').options(driver='com.mysql.jdbc.Driver', url=jdbc_url, dbtable=query).load()
df1.show()

Py4JJavaError: An error occurred while calling o31.load.
: java.lang.ClassNotFoundException: com.mysql.jdbc.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:102)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:38)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:32)
	at org.apache.spark.sql.ex

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
if __name__ == '__main__':
    scSpark = SparkSession \
        .builder \
        .appName("reading csv") \
        .getOrCreate()

In [ ]:
data_file = '12_19_stt.csv'
sdfData = scSpark.read.csv(data_file, header=True, sep=",", encoding='euc-kr').cache()
print('Total Records = {}'.format(sdfData.count()))
sdfData.show()

In [ ]:
type(sdfData)

In [ ]:
pwd

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.config("spark.jars", "/usr/share/java/mysql-connector-java-8.0.22.jar") \
    .master("local").appName("PySpark_MySQL_test").getOrCreate()

wine_df = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/testDB") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "testtest") \
    .option("user", "root").option("password", "root").load()

In [ ]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='root',
                              host='127.0.0.1',
                              database='testdb')

In [ ]:
import findspark

findspark.add_packages('mysql:mysql-connector-java:8.0.11')



from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName('test') \
        .master('local[*]') \
        .config("spark.driver.extraClassPath", "<path to mysql-connector-java-5.1.49-bin.jar>") \
        .getOrCreate()

df = spark.read.format("jdbc").option("url","jdbc:mysql://localhost/testdb").option("driver","com.mysql.jdbc.Driver").option("dbtable","testtest").option("user","root").option("password","root").load()